In [1]:
from noise import *
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.fft import fft, ifft
%matplotlib widget

In [2]:
starti = 38
endi = 42

In [3]:
counters = ['bardeen',
 'einstein',
 'feynman',
 'meitner',
 'newton',
 'noether',
 'rossi',
 'rubin']
# counters = ['bell','curie','feynman','newton','noether','rossi','rubin']
noise_open_dict = {}
noise_closed_dict = {}
fadc_open_dict = {}
fadc_closed_dict = {}
diff_dict = {}
removed_murmur_dict = {}
freq = freq_mhz()
pos_freq = freq[freq>0]
date = '20200615'
openfile =  f'{date}050402.bg.bin'
closedfile =f'{date}050320.bg.bin'
for counter in counters:
    p_open = Path(f'/home/isaac/niche_data/{date}/{counter}/{openfile}')
    p_closed = Path(f'/home/isaac/niche_data/{date}/{counter}/{closedfile}')
    noise_open = read_noise_file(p_open).astype('float')
    # noise_open -= noise_open.mean()
    noise_closed = read_noise_file(p_closed).astype('float')
    # noise_closed -= noise_closed.mean()
    fadc_open_dict[counter] = noise_open
    fadc_closed_dict[counter] = noise_closed
    noise_open_dict[counter] = noise_fft(noise_open)**2
    noise_closed_dict[counter] = noise_fft(noise_closed)**2
    diff_dict[counter] = noise_open_dict[counter] - noise_closed_dict[counter]
    removed_murmur = diff_dict[counter][freq>0.]
    removed_murmur[:starti] = removed_murmur[starti:endi].mean()
    removed_murmur_dict[counter] = np.concatenate(([diff_dict[counter][0]],removed_murmur,[diff_dict[counter][512]],removed_murmur[::-1]))

FileNotFoundError: [Errno 2] No such file or directory: '/home/isaac/niche_data/20200615/rubin/20200615050320.bg.bin'

In [ ]:
def spectrum_fit(freq: float | np.ndarray, scale: float, freq_corner: float, power:float) -> float | np.ndarray:
    return scale * (1 + ((freq+freq_corner)/freq_corner)) ** (-power)
    # return scale * np.exp(-power*(freq/freq_corner))

plt.figure()
plt.plot(pos_freq, spectrum_fit(pos_freq,70,500,10))
plt.loglog()

In [ ]:


for counter in counters:
    plt.figure()
    # guess = (diff_dict[counter][freq>0.].max(),500,5)
    # popt, pcov = curve_fit(spectrum_fit, pos_freq[:endi], removed_murmur[:endi],p0=guess)
    # plt.plot(pos_freq, spectrum_fit(pos_freq,*popt), label = 'fit')
    print(np.sqrt(diff_dict[counter][freq>0.].max()/250))
    plt.plot(freq[freq>0.], noise_open_dict[counter][freq>0.], label = 'open')
    plt.plot(freq[freq>0.], noise_closed_dict[counter][freq>0.], label = 'closed')
    plt.plot(freq[freq>0.], diff_dict[counter][freq>0.], label = 'diff')
    plt.plot(freq[freq>0.], removed_murmur_dict[counter][freq>0.], label = 'removed murmur')
    plt.legend()
    plt.title(counter)
    plt.loglog()
    plt.grid()
    plt.xlabel('Frequency (MHz)')
    plt.ylabel('Relative Power')

In [ ]:
plt.figure()
plt.plot(diff_dict['newton'][1:])
plt.plot(removed_murmur_dict['newton'][1:])

In [ ]:
# gains = {'bardeen': 2.1882036884651583,
#  'bell': 7.406880637204029,
#  'curie': 9.561962861718685,
#  'feynman': 7.734872492963433,
#  'newton': 11.281268225032628,
#  'noether': 9.834954480425155,
#  'rossi': 5.1908236939723436,
#  'rubin': 5.783684658654795,
#  'rutherford': 8.852228215755146}
# gains = {'bardeen': 10.384855151751086,
#  'bell': 35.3473337231697,
#  'curie': 45.38156687381197,
#  'feynman': 36.2549482800708,
#  'newton': 52.702193982134844,
#  'noether': 45.322859969627345,
#  'rossi': 24.25794714099332,
#  'rubin': 27.323269274568446}

gains = {'bardeen': 0.4301294450523357,
 'bell': 1.4639139998316604,
 'curie': 1.8795107279931322,
 'feynman': 1.5014120362528374,
 'newton': 2.182147680064487,
 'noether': 1.8765021501429047,
 'rossi': 1.0044919895844284,
 'rubin': 1.131329751694293}

In [ ]:
plt.figure()
for counter in counters:
    plt.scatter(0,diff_dict[counter][freq>0.].max()/gains[counter]**2,label=counter)
plt.legend()

In [ ]:
plt.figure()
for counter in counters:
    nsbg = (np.var(fadc_open_dict[counter]) - np.var(fadc_closed_dict[counter]))/gains[counter]
    plt.scatter(0,nsbg,label=counter)
plt.legend()

In [ ]:
plt.figure()
for counter in counters:
    plt.scatter(0,removed_murmur_dict[counter][1],label=counter)
plt.legend()

In [ ]:
nsbg = np.array([diff_dict[counter][freq>0.].max()/gains[counter]**2 for counter in counters])

In [ ]:
nsbg.mean()

In [ ]:
A = .0077 #m^2
niche_omega =4*np.pi*np.sin(np.radians(45/2))**2

In [ ]:
niche_omega

In [ ]:
np.mean(nsbg*5.*A*niche_omega)

In [ ]:
(3/.27/.9/.9)*100*4*(4*np.pi*np.sin(np.radians(.5))**2)

In [ ]:
thetac = np.arctan(np.sqrt(77/np.pi)/62.)
frac = (1-np.cos(thetac)**5)

def e_output(amp: float | np.ndarray, a: float, b: float) -> float | np.ndarray:
    return a * (1 - np.exp(-amp/b))
planck = 6.62607015e-34
def nphotons(amp: float | np.ndarray) -> float | np.ndarray:
    return 1.e-12 * e_output(amp,488.095,211.949) / (planck * (3e8/355e-9))

amps = np.linspace(0,120,100)
plt.figure()
plt.plot(amps,frac * .005 * nphotons(amps))